In [1]:
!pip install pandas==1.0.3

# LOAD PACKAGE AND CODE

In [2]:
import numpy as np 
import pandas as pd 
from pykalman import KalmanFilter
from tqdm.notebook import tqdm
from scipy import signal
import gc
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
lab = {1: [0, 1], 2: [0, 1], 3: [0, 1, 2, 3], 5: [0, 1, 2, 3, 4, 5]}
missing = {}
for i, l in lab.items():
    missing[i] = min([x for x in range(10) if x not in l])

missing

{1: 2, 2: 2, 3: 4, 5: 6}

In [4]:
f1_dict = {}
f1_group = pd.DataFrame({'Group': [1, 2, 3, 5, 10]})

In [5]:
path = '../input/lgbm-dataset/'

group = np.load(path + 'group.npy', allow_pickle = True)
train_target = pd.read_csv('../input/liverpool-ion-switching/train.csv', usecols = ['open_channels'])
time = pd.read_csv('../input/liverpool-ion-switching/train.csv', usecols = ['open_channels', 'time'])

train = pd.read_csv(path + 'train_clean.csv')
test = pd.read_csv(path + 'test_clean.csv', usecols = ['segment'])
gc.collect()

0

In [6]:
out_ = np.arange(3640000, 3824000)
idx = train.index
bool_mask = idx.isin(out_)

train = train[~idx.isin(out_)].reset_index(drop = True)
group = group[~idx.isin(out_)]
train_target = train_target[~idx.isin(out_)].reset_index(drop = True)


# WAVENET

In [7]:
gc.collect()

pred_wave = np.load('../input/nnet-composer/pred_wavenet_no_out_oof.npy', allow_pickle = True)
pred_label_wave = pred_wave.argmax(axis = 1)

In [8]:
info_wave = pd.DataFrame({'group': train.segment.values.reshape((-1)), 'pred': pred_label_wave.astype(np.int64), 'target': train_target.values.reshape((-1))})

In [9]:
group = []
label = []
for i in np.unique(train.segment.values):
    temp = info_wave[info_wave['group'] == i]
    for j in np.unique(temp.target):
        group += [i]
        label += [j]
        
f1_intra = pd.DataFrame({'group': group, 'label': label})


# F1 ALL

In [10]:
f1_ = f1_score(info_wave.target, info_wave.pred, average='macro')
f1_dict['wave'] = f1_
print(f'ALL VALUES F1 SCORE OOF: {f1_}')

ALL VALUES F1 SCORE OOF: 0.9415073056228973


# F1 GROUP

In [11]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_wave[info_wave['group'] == i]
    for j in np.unique(temp.target):
        f1 = f1_score(temp.target==j, temp.pred==j)
        l_ += [f1]
        print(f'Group: {i} - LABEL {j}; F1 SCORE:  {f1}')
       
f1_intra['wave'] = l_

Group: 1 - LABEL 0; F1 SCORE:  0.9998217243402245
Group: 1 - LABEL 1; F1 SCORE:  0.9951421834606972
Group: 2 - LABEL 0; F1 SCORE:  0.9956140702872323
Group: 2 - LABEL 1; F1 SCORE:  0.9985442078652734
Group: 3 - LABEL 0; F1 SCORE:  0.9789264813000778
Group: 3 - LABEL 1; F1 SCORE:  0.9832026564211332
Group: 3 - LABEL 2; F1 SCORE:  0.9867702874177289
Group: 3 - LABEL 3; F1 SCORE:  0.9909679280923607
Group: 5 - LABEL 0; F1 SCORE:  0.9588541666666667
Group: 5 - LABEL 1; F1 SCORE:  0.967007721597073
Group: 5 - LABEL 2; F1 SCORE:  0.973235622112474
Group: 5 - LABEL 3; F1 SCORE:  0.977431466616108
Group: 5 - LABEL 4; F1 SCORE:  0.980625959342593
Group: 5 - LABEL 5; F1 SCORE:  0.9834695858121787
Group: 10 - LABEL 0; F1 SCORE:  0.0
Group: 10 - LABEL 1; F1 SCORE:  0.8
Group: 10 - LABEL 2; F1 SCORE:  0.8271662763466043
Group: 10 - LABEL 3; F1 SCORE:  0.8561744768641321
Group: 10 - LABEL 4; F1 SCORE:  0.8683530187053277
Group: 10 - LABEL 5; F1 SCORE:  0.8807414923081142
Group: 10 - LABEL 6; F1 SCOR

# F1 GROUP - CORRECTED

In [12]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_wave[info_wave['group'] == i]
    if i == 1:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 2:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 3:
        temp = temp[(temp.pred.isin([0, 1, 2, 3]))]
    if i == 5:
        temp = temp[(temp.pred.isin([0, 1, 2, 3, 4, 5]))]
    if i == 10:
        temp = temp[(temp.target!=0)]

    f1 = f1_score(temp.target, temp.pred, average='macro')
    l_ += [f1]
    print(f'Group: {i}; F1 SCORE:  {f1}')

f1_group['wave'] = l_

Group: 1; F1 SCORE:  0.9974824720518047
Group: 2; F1 SCORE:  0.9970791390762528
Group: 3; F1 SCORE:  0.9849680851324858
Group: 5; F1 SCORE:  0.973437420357849
Group: 10; F1 SCORE:  0.7922232675878423


# BILSTM NOOUT

In [13]:
pred_bilstm = np.load('../input/nnet-composer/pred_bilstm_noout_oof.npy', allow_pickle = True)

pred_label_bilstm = pred_bilstm.argmax(axis = 1)
info_bilstm = pd.DataFrame({'group': train.segment.values.reshape((-1)), 'pred': pred_label_bilstm.astype(np.int64), 'target': train_target.values.reshape((-1))})

# F1

In [14]:
f1_ = f1_score(info_bilstm.target, info_bilstm.pred, average='macro')
f1_dict['bilstm'] = f1_
print(f'ALL VALUES F1 SCORE OOF: {f1_}')

ALL VALUES F1 SCORE OOF: 0.9414586823166506


# F1 GROUP

In [15]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_bilstm[info_bilstm['group'] == i]
    for j in np.unique(temp.target):
        f1 = f1_score(temp.target==j, temp.pred==j)
        l_ += [f1]
        print(f'Group: {i} - LABEL {j}; F1 SCORE:  {f1}')

f1_intra['bilstm'] = l_

Group: 1 - LABEL 0; F1 SCORE:  0.9998222384378542
Group: 1 - LABEL 1; F1 SCORE:  0.995145355439473
Group: 2 - LABEL 0; F1 SCORE:  0.9957070002589045
Group: 2 - LABEL 1; F1 SCORE:  0.9985756588826213
Group: 3 - LABEL 0; F1 SCORE:  0.9781575970843283
Group: 3 - LABEL 1; F1 SCORE:  0.9833955839688688
Group: 3 - LABEL 2; F1 SCORE:  0.9869899943668017
Group: 3 - LABEL 3; F1 SCORE:  0.9911004715659646
Group: 5 - LABEL 0; F1 SCORE:  0.9563398692810456
Group: 5 - LABEL 1; F1 SCORE:  0.9671487120681235
Group: 5 - LABEL 2; F1 SCORE:  0.9736476649590723
Group: 5 - LABEL 3; F1 SCORE:  0.9777651497854651
Group: 5 - LABEL 4; F1 SCORE:  0.9808218445010803
Group: 5 - LABEL 5; F1 SCORE:  0.9836314124190508
Group: 10 - LABEL 0; F1 SCORE:  0.8
Group: 10 - LABEL 1; F1 SCORE:  0.6948356807511737
Group: 10 - LABEL 2; F1 SCORE:  0.8250701590271282
Group: 10 - LABEL 3; F1 SCORE:  0.8601788069764034
Group: 10 - LABEL 4; F1 SCORE:  0.8696782470711368
Group: 10 - LABEL 5; F1 SCORE:  0.8808467841656638
Group: 10 

# F1 GROUP CORRECTED

In [16]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_bilstm[info_bilstm['group'] == i]
    if i == 1:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 2:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 3:
        temp = temp[(temp.pred.isin([0, 1, 2, 3]))]
    if i == 5:
        temp = temp[(temp.pred.isin([0, 1, 2, 3, 4, 5]))]
    if i == 10:
        temp = temp[(temp.target!=0)]

    f1 = f1_score(temp.target, temp.pred, average='macro')
    l_ += [f1]
    print(f'Group: {i}; F1 SCORE:  {f1}')
    
f1_group['bilstm'] = l_

Group: 1; F1 SCORE:  0.9974840560203766
Group: 2; F1 SCORE:  0.997141329570763
Group: 3; F1 SCORE:  0.9849109117464909
Group: 5; F1 SCORE:  0.9732257755023063
Group: 10; F1 SCORE:  0.7821709890469044


# LGBM

In [17]:
pred_lgbm = np.load('../input/lgbm-kfold-no-out/pred_oof.npy', allow_pickle = True)

pred_label_lgbm = pred_lgbm.argmax(axis = 1)
info_lgbm = pd.DataFrame({'group': train.segment.values.reshape((-1)), 'pred': pred_label_lgbm.astype(np.int64), 'target': train_target.values.reshape((-1))})

# F1

In [18]:
f1_ = f1_score(info_lgbm.target, info_lgbm.pred, average='macro')
f1_dict['lgbm'] = f1_
print(f'ALL VALUES F1 SCORE OOF: {f1_}')

ALL VALUES F1 SCORE OOF: 0.9405856517188351


# F1 GROUP

In [19]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_lgbm[info_lgbm['group'] == i]
    for j in np.unique(temp.target):
        f1 = f1_score(temp.target==j, temp.pred==j)
        l_ += [f1]
        print(f'Group: {i} - LABEL {j}; F1 SCORE:  {f1}')

f1_intra['lgbm'] = l_

Group: 1 - LABEL 0; F1 SCORE:  0.9998113543925404
Group: 1 - LABEL 1; F1 SCORE:  0.994847775175644
Group: 2 - LABEL 0; F1 SCORE:  0.9954526663910708
Group: 2 - LABEL 1; F1 SCORE:  0.9984910294162694
Group: 3 - LABEL 0; F1 SCORE:  0.9777397260273972
Group: 3 - LABEL 1; F1 SCORE:  0.9826227731302631
Group: 3 - LABEL 2; F1 SCORE:  0.9864106614268594
Group: 3 - LABEL 3; F1 SCORE:  0.990756975052725
Group: 5 - LABEL 0; F1 SCORE:  0.9574579831932775
Group: 5 - LABEL 1; F1 SCORE:  0.9672852496651999
Group: 5 - LABEL 2; F1 SCORE:  0.9728305338642744
Group: 5 - LABEL 3; F1 SCORE:  0.9772355716804764
Group: 5 - LABEL 4; F1 SCORE:  0.980449031415609
Group: 5 - LABEL 5; F1 SCORE:  0.9832054336310788
Group: 10 - LABEL 0; F1 SCORE:  0.0
Group: 10 - LABEL 1; F1 SCORE:  0.6354515050167224
Group: 10 - LABEL 2; F1 SCORE:  0.8188235294117647
Group: 10 - LABEL 3; F1 SCORE:  0.8557126030624264
Group: 10 - LABEL 4; F1 SCORE:  0.8676269901440485
Group: 10 - LABEL 5; F1 SCORE:  0.8799364714258332
Group: 10 - 

# F1 GROUP CORRECTED

In [20]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_lgbm[info_lgbm['group'] == i]
    if i == 1:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 2:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 3:
        temp = temp[(temp.pred.isin([0, 1, 2, 3]))]
    if i == 5:
        temp = temp[(temp.pred.isin([0, 1, 2, 3, 4, 5]))]
    if i == 10:
        temp = temp[(temp.target!=0)]

    f1 = f1_score(temp.target, temp.pred, average='macro')
    l_ += [f1]
    print(f'Group: {i}; F1 SCORE:  {f1}')

f1_group['lgbm'] = l_

Group: 1; F1 SCORE:  0.9973298238639248
Group: 2; F1 SCORE:  0.9969718479036701
Group: 3; F1 SCORE:  0.9843825339093112
Group: 5; F1 SCORE:  0.9730773005749859
Group: 10; F1 SCORE:  0.7752811091863983


# ENSEMBLE 2

In [21]:
pred_ens2 = pred_bilstm + pred_wave 
pred_label_ens2 = pred_ens2.argmax(axis = 1)

In [22]:
info_ens2 = pd.DataFrame({'group': train.segment.values.reshape((-1)), 'pred': pred_label_ens2.astype(np.int64), 'target': train_target.values.reshape((-1))})

# F1 ALL

In [23]:
f1_ = f1_score(info_ens2.target, info_ens2.pred, average='macro')
f1_dict['ens2'] = f1_
print(f'ALL VALUES F1 SCORE OOF: {f1_}')

ALL VALUES F1 SCORE OOF: 0.9418560284068126


# GROUP

In [24]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_ens2[info_ens2['group'] == i]
    for j in np.unique(temp.target):
        f1 = f1_score(temp.target==j, temp.pred==j)
        l_ += [f1]
        print(f'Group: {i} - LABEL {j}; F1 SCORE:  {f1}')
       
f1_intra['ens2'] = l_

Group: 1 - LABEL 0; F1 SCORE:  0.9998217224924024
Group: 1 - LABEL 1; F1 SCORE:  0.9951435630910795
Group: 2 - LABEL 0; F1 SCORE:  0.9956714647787269
Group: 2 - LABEL 1; F1 SCORE:  0.9985636078016673
Group: 3 - LABEL 0; F1 SCORE:  0.9790839157323143
Group: 3 - LABEL 1; F1 SCORE:  0.9836204589607872
Group: 3 - LABEL 2; F1 SCORE:  0.9870665194792729
Group: 3 - LABEL 3; F1 SCORE:  0.9911389219748696
Group: 5 - LABEL 0; F1 SCORE:  0.9581808677469944
Group: 5 - LABEL 1; F1 SCORE:  0.9676665533650578
Group: 5 - LABEL 2; F1 SCORE:  0.9738619110331929
Group: 5 - LABEL 3; F1 SCORE:  0.9778870444035729
Group: 5 - LABEL 4; F1 SCORE:  0.980930868883174
Group: 5 - LABEL 5; F1 SCORE:  0.9837058228828324
Group: 10 - LABEL 0; F1 SCORE:  0.5
Group: 10 - LABEL 1; F1 SCORE:  0.7727272727272727
Group: 10 - LABEL 2; F1 SCORE:  0.829976580796253
Group: 10 - LABEL 3; F1 SCORE:  0.8611151978814183
Group: 10 - LABEL 4; F1 SCORE:  0.869991050169709
Group: 10 - LABEL 5; F1 SCORE:  0.8811767031229412
Group: 10 - 

# GROUP CORRECTED

In [25]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_ens2[info_ens2['group'] == i]
    if i == 1:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 2:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 3:
        temp = temp[(temp.pred.isin([0, 1, 2, 3]))]
    if i == 5:
        temp = temp[(temp.pred.isin([0, 1, 2, 3, 4, 5]))]
    if i == 10:
        temp = temp[(temp.target!=0)]

    f1 = f1_score(temp.target, temp.pred, average='macro')
    l_ += [f1]
    print(f'Group: {i}; F1 SCORE:  {f1}')
    
f1_group['ens2'] = l_

Group: 1; F1 SCORE:  0.9974831609484546
Group: 2; F1 SCORE:  0.9971175362901972
Group: 3; F1 SCORE:  0.9852278697209295
Group: 5; F1 SCORE:  0.9737055113858041
Group: 10; F1 SCORE:  0.7905113541893655


# ENSEMBLE 3

In [26]:
pred_ens3 = pred_bilstm + pred_wave + pred_lgbm
pred_label_ens3 = pred_ens3.argmax(axis = 1)

In [27]:
info_ens = pd.DataFrame({'group': train.segment.values.reshape((-1)), 'pred': pred_label_ens3.astype(np.int64), 'target': train_target.values.reshape((-1))})

# F1 ALL

In [28]:
f1_ = f1_score(info_ens.target, info_ens.pred, average='macro')
f1_dict['ens'] = f1_
print(f'ALL VALUES F1 SCORE OOF: {f1_}')

ALL VALUES F1 SCORE OOF: 0.941390122161667


# GROUP

In [29]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_ens[info_ens['group'] == i]
    for j in np.unique(temp.target):
        f1 = f1_score(temp.target==j, temp.pred==j)
        l_ += [f1]
        print(f'Group: {i} - LABEL {j}; F1 SCORE:  {f1}')
       
f1_intra['ens3'] = l_

Group: 1 - LABEL 0; F1 SCORE:  0.9998123926569865
Group: 1 - LABEL 1; F1 SCORE:  0.9948748527052544
Group: 2 - LABEL 0; F1 SCORE:  0.9956152233025295
Group: 2 - LABEL 1; F1 SCORE:  0.9985449678194829
Group: 3 - LABEL 0; F1 SCORE:  0.9789129340714645
Group: 3 - LABEL 1; F1 SCORE:  0.9832587424250848
Group: 3 - LABEL 2; F1 SCORE:  0.986842458163647
Group: 3 - LABEL 3; F1 SCORE:  0.9910393874370231
Group: 5 - LABEL 0; F1 SCORE:  0.9593709043250328
Group: 5 - LABEL 1; F1 SCORE:  0.9676733756987099
Group: 5 - LABEL 2; F1 SCORE:  0.973497958961063
Group: 5 - LABEL 3; F1 SCORE:  0.9776929919803009
Group: 5 - LABEL 4; F1 SCORE:  0.98078800996316
Group: 5 - LABEL 5; F1 SCORE:  0.9835370919562172
Group: 10 - LABEL 0; F1 SCORE:  0.0
Group: 10 - LABEL 1; F1 SCORE:  0.8051948051948051
Group: 10 - LABEL 2; F1 SCORE:  0.8365520489872822
Group: 10 - LABEL 3; F1 SCORE:  0.8602103095815867
Group: 10 - LABEL 4; F1 SCORE:  0.8693058294150361
Group: 10 - LABEL 5; F1 SCORE:  0.8806297417006533
Group: 10 - L

# GROUP - CORRECTED

In [30]:
l_ = []
for i in np.unique(train.segment.values):
    temp = info_ens[info_ens['group'] == i]
    if i == 1:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 2:
        temp = temp[(temp.pred.isin([0, 1]))]
    if i == 3:
        temp = temp[(temp.pred.isin([0, 1, 2, 3]))]
    if i == 5:
        temp = temp[(temp.pred.isin([0, 1, 2, 3, 4, 5]))]
    if i == 10:
        temp = temp[(temp.target!=0)]

    f1 = f1_score(temp.target, temp.pred, average='macro')
    l_ += [f1]
    print(f'Group: {i}; F1 SCORE:  {f1}')
    
f1_group['ens3'] = l_

Group: 1; F1 SCORE:  0.9973438817588051
Group: 2; F1 SCORE:  0.9970800955610062
Group: 3; F1 SCORE:  0.9850133805243049
Group: 5; F1 SCORE:  0.9737600554807472
Group: 10; F1 SCORE:  0.7938118022959709


# FINAL COMPARISON

In [31]:
f1_df = pd.DataFrame({'Name': list(f1_dict.keys()), 'F1': list(f1_dict.values())})
print(f1_df.sort_values('F1', ascending = False))


     Name        F1
3    ens2  0.941856
0    wave  0.941507
1  bilstm  0.941459
4     ens  0.941390
2    lgbm  0.940586


In [32]:
print(f1_group.to_string())


   Group      wave    bilstm      lgbm      ens2      ens3
0      1  0.997482  0.997484  0.997330  0.997483  0.997344
1      2  0.997079  0.997141  0.996972  0.997118  0.997080
2      3  0.984968  0.984911  0.984383  0.985228  0.985013
3      5  0.973437  0.973226  0.973077  0.973706  0.973760
4     10  0.792223  0.782171  0.775281  0.790511  0.793812


In [33]:
print(f1_intra.to_string())

    group  label      wave    bilstm      lgbm      ens2      ens3
0       1      0  0.999822  0.999822  0.999811  0.999822  0.999812
1       1      1  0.995142  0.995145  0.994848  0.995144  0.994875
2       2      0  0.995614  0.995707  0.995453  0.995671  0.995615
3       2      1  0.998544  0.998576  0.998491  0.998564  0.998545
4       3      0  0.978926  0.978158  0.977740  0.979084  0.978913
5       3      1  0.983203  0.983396  0.982623  0.983620  0.983259
6       3      2  0.986770  0.986990  0.986411  0.987067  0.986842
7       3      3  0.990968  0.991100  0.990757  0.991139  0.991039
8       5      0  0.958854  0.956340  0.957458  0.958181  0.959371
9       5      1  0.967008  0.967149  0.967285  0.967667  0.967673
10      5      2  0.973236  0.973648  0.972831  0.973862  0.973498
11      5      3  0.977431  0.977765  0.977236  0.977887  0.977693
12      5      4  0.980626  0.980822  0.980449  0.980931  0.980788
13      5      5  0.983470  0.983631  0.983205  0.983706  0.98

# FINAL PREDICTION

In [34]:
pred_lgb_test = np.load('../input/lgbm-kfold-no-out/pred_test.npy', allow_pickle = True)
pred_wave_test = np.load('../input/nnet-composer/pred_wavenet_no_out.npy', allow_pickle = True).reshape((-1, 11))
pred_bilstm_test = np.load('../input/nnet-composer/pred_bilstm_noout.npy', allow_pickle = True)
# pred_xgb_test = np.load('../input/xgboost-kf/xgboost-kf/xgboost_pred.npy', allow_pickle = True)

pred_test = pred_wave_test + pred_bilstm_test + pred_lgb_test

In [35]:
pred_test_stack = pd.DataFrame({
#     'xgb': pred_xgb_test.argmax(axis = 1).tolist(),
    'lgb': pred_lgb_test.argmax(axis = 1).tolist(),
    'bilstm': pred_bilstm_test.argmax(axis = 1).tolist(),
    'wavenet': pred_wave_test.argmax(axis = 1).tolist(),
    'ens': pred_test.argmax(axis = 1).tolist()}
)

# CORRECTNESS OF TEST

In [36]:
print('ENSEMBLE\n')
for i in np.unique(test.segment):
    print(f'Group: {i}')
    print(pd.crosstab(pred_test_stack.wavenet[test.segment == i], pred_test_stack.bilstm[test.segment == i]).to_string())
    print('\n\n')

ENSEMBLE

Group: 1
bilstm         0       1     2    3
wavenet                            
0        1189884     133     0    0
1             82  103284     0    0
2              0     877  5294    4
3              0       0     7  428
4              0       0     0    7



Group: 2
bilstm       0      1  3
wavenet                 
0        25987     22  0
1           16  73974  0
3            0      0  1



Group: 3
bilstm      0      1      2      3  5
wavenet                              
0        4235     29      0      0  0
1          15  33999     83      0  0
2           0     88  86736    116  0
3           0      0     98  74599  0
4           0      0      0      1  0
5           0      0      0      0  1



Group: 5
bilstm     0     1      2      3      4      5
wavenet                                       
0        405     6      0      0      0      0
1          3  5134     23      0      0      0
2          0    45  23930     97      0      0
3          0     0     45  58

In [37]:
print('WAVENET\n')
for i in np.unique(test.segment):
    print(f'Group: {i}')
    print(pd.crosstab(pred_test_stack.wavenet[test.segment == i], pred_test_stack.bilstm[test.segment == i]).to_string())
    print('\n\n')

WAVENET

Group: 1
bilstm         0       1     2    3
wavenet                            
0        1189884     133     0    0
1             82  103284     0    0
2              0     877  5294    4
3              0       0     7  428
4              0       0     0    7



Group: 2
bilstm       0      1  3
wavenet                 
0        25987     22  0
1           16  73974  0
3            0      0  1



Group: 3
bilstm      0      1      2      3  5
wavenet                              
0        4235     29      0      0  0
1          15  33999     83      0  0
2           0     88  86736    116  0
3           0      0     98  74599  0
4           0      0      0      1  0
5           0      0      0      0  1



Group: 5
bilstm     0     1      2      3      4      5
wavenet                                       
0        405     6      0      0      0      0
1          3  5134     23      0      0      0
2          0    45  23930     97      0      0
3          0     0     45  582

In [38]:
print('LGB\n')
for i in np.unique(test.segment):
    print(f'Group: {i}')
    print(pd.crosstab(pred_test_stack.lgb[test.segment == i], pred_test_stack.bilstm[test.segment == i]).to_string())
    print('\n\n')

LGB

Group: 1
bilstm        0       1     2    3
lgb                               
0       1189844      94     0    0
1           122  104014   534    0
2             0     186  4643    2
3             0       0   123  410
4             0       0     1   27



Group: 2
bilstm      0      1  3
lgb                    
0       25966     24  0
1          37  73972  1



Group: 3
bilstm     0      1      2      3  5
lgb                                 
0       4211     24      0      0  0
1         39  33953    126      0  0
2          0    139  86633    163  0
3          0      0    158  74553  1



Group: 5
bilstm    0     1      2      3      4      5
lgb                                          
0       398     6      0      0      0      0
1        10  5132     30      0      0      0
2         0    47  23880    112      0      0
3         0     0     88  58279    249      0
4         0     0      0    157  73801    171
5         0     0      0      0    111  37528
9         0     0  

In [39]:
print('ENS\n')
for i in np.unique(test.segment):
    print(f'Group: {i}')
    print(pd.crosstab(pred_test_stack.ens[test.segment == i], pred_test_stack.wavenet[test.segment == i]).to_string())
    print('\n\n')

ENS

Group: 1
wavenet        0       1     2    3  4
ens                                   
0        1189909      41     0    0  0
1            108  103325   382    0  0
2              0       0  5790    3  0
3              0       0     3  432  3
4              0       0     0    0  4



Group: 2
wavenet      0      1  3
ens                     
0        25983     12  0
1           26  73978  0
3            0      0  1



Group: 3
wavenet     0      1      2      3  4  5
ens                                     
0        4239      4      0      0  0  0
1          25  34035     56      0  0  0
2           0     58  86808     64  0  0
3           0      0     76  74633  1  0
5           0      0      0      0  0  1



Group: 5
wavenet    0     1      2      3      4      5
ens                                           
0        402     5      0      0      0      0
1          9  5143     19      0      0      0
2          0    12  23989     35      0      0
3          0     0     64  583

In [40]:
sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})

sub['open_channels'] = pred_test.argmax(axis = 1)
sub.to_csv('submission.csv', index=False)